In [56]:
import os
import torch
import torchvision.transforms as T
import torchvision.models.segmentation as seg
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from scipy import ndimage

In [87]:
source_path = '/Users/aravdhoot/PD/frames'
destination_path = '/Users/aravdhoot/PD/segmented_images'

In [89]:
os.makedirs(destination_path, exist_ok=True)
for coded_folder in os.listdir(source_path):
    if not os.path.isdir(os.path.join(source_path, coded_folder)): os.remove(os.path.join(source_path, coded_folder))
    os.makedirs(os.path.join(destination_path, coded_folder), exist_ok=True)
    for severity in os.listdir(os.path.join(source_path, coded_folder)):
        if not os.path.isdir(os.path.join(source_path, coded_folder, severity)): os.remove(os.path.join(source_path, coded_folder, severity))
        os.makedirs(os.path.join(destination_path, coded_folder, severity), exist_ok=True)
        for video in os.listdir(os.path.join(source_path, coded_folder, severity)):
            if not os.path.isdir(os.path.join(source_path, coded_folder, severity, video)): os.remove(os.path.join(source_path, coded_folder, severity, video))
            os.makedirs(os.path.join(destination_path, coded_folder, severity, video), exist_ok=True)
            for image in os.listdir(os.path.join(source_path, coded_folder, severity, video)):
                full_segmentation(os.path.join(source_path, coded_folder, severity, video, image), os.path.join(destination_path, coded_folder, severity, video, image))

Output predictions shape: (360, 640)
Main object mask shape: (360, 640)
Output predictions shape: (360, 640)
Main object mask shape: (360, 640)
Output predictions shape: (360, 640)
Main object mask shape: (360, 640)
Output predictions shape: (360, 640)
Main object mask shape: (360, 640)
Output predictions shape: (360, 640)
Main object mask shape: (360, 640)
Output predictions shape: (360, 640)
Main object mask shape: (360, 640)
Output predictions shape: (360, 640)
Main object mask shape: (360, 640)
Output predictions shape: (360, 640)
Main object mask shape: (360, 640)
Output predictions shape: (360, 640)
Main object mask shape: (360, 640)
Output predictions shape: (360, 640)
Main object mask shape: (360, 640)
Output predictions shape: (360, 640)
Main object mask shape: (360, 640)
Output predictions shape: (360, 640)
Main object mask shape: (360, 640)
Output predictions shape: (360, 640)
Main object mask shape: (360, 640)
Output predictions shape: (360, 640)
Main object mask shape: (36

KeyboardInterrupt: 

In [22]:
model = seg.deeplabv3_resnet50(pretrained=True, progress=True)
model.eval()

/Users/aravdhoot/opt/anaconda3/envs/ml_env/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DeepLabV3_ResNet50_Weights.COCO_WITH_VOC_LABELS_V1`. You can also use `weights=DeepLabV3_ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


DeepLabV3(
  (backbone): IntermediateLayerGetter(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Se

In [35]:
def preprocess_image(image_path):
    image = Image.open(image_path).convert('RGB')
    transform = T.Compose([
        T.ToTensor(),
        T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    input_tensor = transform(image).unsqueeze(0)
    return input_tensor

In [36]:
def remove_secondary_object(segmentation_mask):
    labeled_array, num_features = ndimage.label(segmentation_mask)
    largest_segment = 0
    max_area = 0
    for feature in range(1, num_features + 1):
        area = np.sum(labeled_array == feature)
        if area > max_area:
            max_area = area
            largest_segment = feature
    main_object_mask = labeled_array == largest_segment
    return main_object_mask.astype(np.uint8)

In [86]:
def full_segmentation(image_path, destination_path):
    input_tensor = preprocess_image(image_path)
    with torch.no_grad():
        output = model(input_tensor)['out'][0]
        output_predictions = output.argmax(0).cpu().numpy()
    main_object_mask = remove_secondary_object(output_predictions)
    print("Output predictions shape:", output_predictions.shape)
    print("Main object mask shape:", main_object_mask.shape)
    main_object_mask_binary = (main_object_mask * 255).astype(np.uint8)
    cv2.imwrite(destination_path, main_object_mask_binary)

In [80]:
full_segmentation('/Users/aravdhoot/PD/frames/MILD-20230609T012657Z-001/MILD/mild_youtube_03032022_06.mp4/0.jpg', 'test.jpg')

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
Output predictions shape: (360, 640)
Main object mask shape: (360, 640)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
